In [11]:
import pandas as pd
import sqlite3
import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import datetime, timezone

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    S = spot price
    K = strike
    T = time to expiration in years
    r = risk-free rate
    sigma = volatility
    """
    if T <= 0 or sigma <= 0:
        return 0  # Option expired or invalid inputs

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == "call":
        return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
def fetch_options_data(ticker: str):
    tick = yf.Ticker(ticker)

    expirations = tick.options
    if not expirations:
        print("No listed options for", ticker)
        return None

    # Choose nearest expiration 
    nearest_exp = expirations[0]

    opt_chain = tick.option_chain(nearest_exp)

    calls = opt_chain.calls
    puts  = opt_chain.puts
    calls["expiration"] = nearest_exp
    puts["expiration"] = nearest_exp

    return calls, puts

def add_bs_prices(options_df, S, r=0.05):
    today = datetime.now(timezone.utc)

    options_df = options_df.copy()
    options_df["bs_price"] = None

    for i, row in options_df.iterrows():
        K = row["strike"]

        # Ensure expiration is timezone-aware (UTC)
        exp_date = datetime.strptime(row["expiration"], "%Y-%m-%d").replace(tzinfo=timezone.utc)

        # Time to expiration in years
        T = (exp_date - today).total_seconds() / (365.25 * 24 * 3600)

        # Get IV safely
        sigma = row.get("impliedVolatility")
        if not sigma or sigma <= 0:
            continue

        # Determine call/put
        option_type = "call" if "C" in row["contractSymbol"] else "put"

        # Compute Black–Scholes
        bs_value = black_scholes(
            S=S, K=K, T=T, r=r, sigma=sigma, option_type=option_type
        )

        options_df.at[i, "bs_price"] = bs_value

    return options_df

def get_latest_close_price(ticker):
    query = """
    SELECT close FROM intraday_prices
    WHERE ticker = ?
    ORDER BY timestamp DESC
    LIMIT 1
    """
    df = pd.read_sql_query(query, conn, params=(ticker,))
    if df.empty:
        return None
    return df["close"].iloc[0]

def get_options_with_bs(ticker):
    spot = get_latest_close_price(ticker)
    if not spot:
        print("No price in DB — fetching now...")
        df = fetch_intraday_data(ticker)
        store_data_to_db(df, get_latest_timestamp(ticker))
        spot = df["close"].iloc[-1]

    calls, puts = fetch_options_data(ticker)

    # Add BS prices
    calls_bs = add_bs_prices(calls, S=spot)
    puts_bs  = add_bs_prices(puts, S=spot)

    # Add type column
    calls_bs["type"] = "call"
    puts_bs["type"]  = "put"

    # Store into DB
    store_options_to_db(calls_bs, ticker)
    store_options_to_db(puts_bs, ticker)

    return calls_bs, puts_bs



def fetch_intraday_data(ticker: str, interval: str = '1m') -> pd.DataFrame:
    tick = yf.Ticker(ticker)

    df = tick.history(interval=interval, period='1d')
    df.drop(columns=['Dividends', 'Stock Splits'], inplace=True)   
    df.columns = ["open", "high", "low", "close", "volume"]
    df.index = df.index.tz_localize(None)       
    df.index = df.index.to_pydatetime()
    df["timestamp"] = df.index.astype(str)
    df["ticker"] = ticker
    df.reset_index(drop=True, inplace=True)

    return df

conn = sqlite3.connect('intraday_stock_prices.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS intraday_prices (
        ticker TEXT,
        timestamp TEXT,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        PRIMARY KEY (ticker, timestamp)
    )
''')

conn.commit()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS options_prices (
        ticker TEXT,
        contractSymbol TEXT PRIMARY KEY,
        expiration TEXT,
        strike REAL,
        type TEXT,
        lastPrice REAL,
        impliedVol REAL,
        bs_price REAL,
        updated_at TEXT
    )
''')
conn.commit()


def get_latest_timestamp(ticker: str):
    cursor.execute("""
        SELECT MAX(timestamp) FROM intraday_prices 
        WHERE ticker = ?
    """, (ticker,))
    return cursor.fetchone()[0]

def store_data_to_db(data: pd.DataFrame, latest_timestamp: str):
    if data is None:
        print("No data to store.")
        return

    if latest_timestamp:
        data = data[data["timestamp"] > latest_timestamp]

    if data.empty:
        print("No new data to insert.")
        return

    try:
        data.to_sql("intraday_prices", conn, if_exists="append", index=False)
    except Exception as e:
        
        print("Insert error:", e)

def store_options_to_db(df: pd.DataFrame, ticker: str):
    if df is None or df.empty:
        return

    df = df.copy()
    df["ticker"] = ticker
    df["updated_at"] = datetime.now(timezone.utc).isoformat()

    keep_cols = [
        "ticker", "contractSymbol", "expiration", "strike",
        "type", "lastPrice", "impliedVolatility", "bs_price", "updated_at"
    ]

    df = df[keep_cols]
    df.rename(columns={"type": "type", "impliedVolatility": "impliedVol"}, inplace=True)

    df.to_sql("options_prices", conn, if_exists="replace", index=False)


def query_data(ticker, start_time, end_time):
    query = """
    SELECT * FROM intraday_prices
    WHERE ticker = ? 
      AND timestamp BETWEEN ? AND ?
    """
    df = pd.read_sql_query(query, conn, params=(ticker, start_time, end_time))
    return df
    
    

In [12]:
calls_bs, puts_bs = get_options_with_bs("AAPL")

print("CALLS (Market vs Black-Scholes):")
print(calls_bs[["contractSymbol", "strike", "lastPrice", "bs_price"]].head())

print("\nPUTS (Market vs Black-Scholes):")
print(puts_bs[["contractSymbol", "strike", "lastPrice", "bs_price"]].head())


CALLS (Market vs Black-Scholes):
        contractSymbol  strike  lastPrice    bs_price
0  AAPL251121C00100000   100.0     169.22  167.533641
1  AAPL251121C00105000   105.0     164.44  162.534822
2  AAPL251121C00110000   110.0     159.30  157.536004
3  AAPL251121C00115000   115.0     154.82  152.537185
4  AAPL251121C00120000   120.0     148.45  147.538367

PUTS (Market vs Black-Scholes):
        contractSymbol  strike  lastPrice bs_price
0  AAPL251121P00100000   100.0       0.01      0.0
1  AAPL251121P00105000   105.0       0.01      0.0
2  AAPL251121P00110000   110.0       0.01      0.0
3  AAPL251121P00115000   115.0       0.01      0.0
4  AAPL251121P00120000   120.0       0.01      0.0


In [13]:
query = """SELECT * FROM intraday_prices
WHERE ticker = 'AAPL'""" 

df = pd.read_sql_query(query, conn)
df.head()

,ticker,timestamp,open,high,low,close,volume
0,AAPL,2025-11-17 09:30:00-05:00,268.720001,269.359985,267.660004,267.940094,3047045
1,AAPL,2025-11-17 09:31:00-05:00,267.941498,268.079987,267.000000,267.459991,381795
2,AAPL,2025-11-17 09:32:00-05:00,267.410004,268.339996,267.299988,268.250000,227990
3,AAPL,2025-11-17 09:33:00-05:00,268.255005,268.769989,267.859985,267.989105,198866
4,AAPL,2025-11-17 09:34:00-05:00,267.880005,267.880005,267.260010,267.549011,173896
